## 1. Libraries and Spark setup


This section imports necessary libraries and sets up the Spark environment:

Libraries:
* `o`, `sys`, `json`, `datetime`, `numpy`, `pandas`, `tqdm`, `matplotlib.pyplot`: General purpose libraries for file system access, system functionalities, JSON handling, date/time manipulation, numerical computation, data manipulation, progress bars, and plotting.

pyspark libraries:
* `SparkContext`, `SparkConf`: Core Spark functionalities for setting up the Spark context and configuration.
* `SparkSession`: Entry point for interacting with Spark SQL.
* `functions as F`: Provides various Spark SQL functions for data manipulation.
* `types`: Defines data types for Spark DataFrames.
* `Window`: Used for window functions in Spark SQL.
* `ml.feature`: Provides feature engineering and transformation tools like `Word2Vec`, `Imputer`, `OneHotEncoder`, `StringIndexer`, `VectorAssembler`.
* `ml.classification`: Provides classification algorithms like `LogisticRegression` and `RandomForestClassifier`.
* `ml.evaluation`: Provides evaluation metrics like `BinaryClassificationEvaluator` and `BinaryClassificationMetrics`.
* `ml.tuning`: Provides tools for hyperparameter tuning like `CrossValidator` and `ParamGridBuilder`.

Spark Configuration:
* `SparkConf`: Sets configuration parameters for the Spark application.
* `spark.master`: Specifies the cluster manager; local[*] indicates using all available cores on the local machine.
* `spark.driver.memory`, `spark.driver.maxResultSize`: Allocates memory for the driver process.
* `SparkContext`, `SparkSession`: Creates the Spark context and session based on the configuration.

Accessing Data:
* `access_data`: Function to load JSON data from a local file.
* `access_s3_data`: Loads AWS credentials from a local JSON file.

Spark configuration is further set to access data from Yandex Cloud Storage (S3-compatible) using the loaded credentials.

In [1]:
!pip install wandb


  Using cached wandb-0.17.0-py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
  Using cached setproctitle-1.3.3-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
  Using cached sentry_sdk-2.4.0-py2.py3-none-any.whl (289 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 1.8.22 requires urllib3<2, but you have urllib3 2.2.1 which is incompatible.
botocore 1.34.84 requires urllib3<1.27,>=1.25.4; python_version < "3.10", but you have urllib3 2.2.1 which is incompatible.


In [2]:
import os
import sys
import json
import datetime
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Create filenames

import datetime
import uuid
import getpass

In [3]:
# Set name of notebook
os.environ["WANDB_NOTEBOOK_NAME"] = "full_pipeline_with_logging_hyperopt_Anna_1month.ipynb"

In [4]:
# Pyspark general
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Pyspark pre-processing
from pyspark.sql.window import Window

# Pyspark vectorization
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

#
from pyspark.ml import Pipeline
from pyspark.ml.feature import Imputer

# Pyspark models
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, LinearSVC
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

# Pyspark classifiers
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

# Pyspark cross-validation
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Pyspark reporting
from sklearn.metrics import classification_report

# Pyspark other
from pyspark.ml.functions import vector_to_array


In [5]:
!pip install hyperopt


  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached future-1.0.0-py3-none-any.whl (491 kB)


In [6]:
# Hyperopt related
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope


In [7]:
# Weights and biases login

wandb.login()

NameError: name 'wandb' is not defined

In [8]:
# W&B logging
import wandb

In [9]:
print('user:', os.environ['JUPYTERHUB_SERVICE_PREFIX'])

def uiWebUrl(self):
    from urllib.parse import urlparse
    web_url = self._jsc.sc().uiWebUrl().get()
    port = urlparse(web_url).port
    return '{}proxy/{}/jobs/'.format(os.environ['JUPYTERHUB_SERVICE_PREFIX'], port)

SparkContext.uiWebUrl = property(uiWebUrl)

conf = SparkConf()
conf.set('spark.master', 'local[*]')
conf.set('spark.driver.memory', '32G')
conf.set('spark.driver.maxResultSize', '8G')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)
spark

user: /user/st110923/


In [10]:
def access_data(file_path):
    with open(file_path) as file:
        access_data = json.load(file)
    return access_data

access_s3_data = access_data('.access_jhub_data')

In [11]:
spark._jsc.hadoopConfiguration().set('fs.s3a.access.key', access_s3_data['aws_access_key_id'])
spark._jsc.hadoopConfiguration().set('fs.s3a.secret.key', access_s3_data['aws_secret_access_key'])
spark._jsc.hadoopConfiguration().set('fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem')
spark._jsc.hadoopConfiguration().set('fs.s3a.multipart.size', '104857600')
spark._jsc.hadoopConfiguration().set('fs.s3a.block.size', '33554432')
spark._jsc.hadoopConfiguration().set('fs.s3a.threads.max', '256')
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint', 'http://storage.yandexcloud.net')
spark._jsc.hadoopConfiguration().set('fs.s3a.aws.credentials.provider',
                                     'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')

v5 = data week
2024, 1, 1, 0, 0, 0
     2024, 4, 16, 23, 59, 59

In [12]:
VER = 'vA1'
PROC_DS = True
PROC_LAGS = True
FRAC_0 = .001 # used only if `PROC_LAGS = True`
PROC_VECS = True
BUCKET = 'pvc-84ea79a0-dc20-4a2d-86ab-f83c1f8d4a7b'
PRJ_PATH = '/home/jovyan/__RAYPFP'

files_path = 'data/events'
files_mask = f'{files_path}/data_202*-*-*.csv'

file_path_ds = f's3a://{BUCKET}/work/{VER}/data_raw.parquet'
file_path_lags = f's3a://{BUCKET}/work/{VER}/data_lags.parquet'
file_path_trn = f's3a://{BUCKET}/work/{VER}/data_vec_train.parquet'
file_path_tst = f's3a://{BUCKET}/work/{VER}/data_vec_test.parquet'
file_path_lags1 = f's3a://{BUCKET}/work/{VER}/data_lags1.parquet'


In [13]:
#file_path_lags1 = f's3a://{BUCKET}/work/{VER}/data_lags1.parquet'


In [14]:
def clean_parquet(path):
    cmd = path.replace(
        f's3a://{BUCKET}',
        f'rm -rf {PRJ_PATH}'
    )
    !{cmd}
    return f'command to run: {cmd}'

In [15]:
# Create filenames that will be used later when saving predictions

current_datetime = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
current_user = os.environ['JUPYTERHUB_SERVICE_PREFIX']
current_user = current_user.split("/")[2]
unique_identifier = str(uuid.uuid4())[:8]  # Generate a unique identifier (first 8 characters)


In [16]:
print("**Filename Information:**")
print(f"- Current Date and Time: {current_datetime}")
print(f"- Current User: {current_user}")
print(f"- Unique Identifier: {unique_identifier}")

**Filename Information:**
- Current Date and Time: 20240605_115859
- Current User: st110923
- Unique Identifier: e2d6ed98


### 2.1. Load or preprocess data - `raw` stage

2.1. Load or Preprocess Data - raw Stage

This section checks the `PROC_DS` flag.

If True, it reads raw CSV data from S3, parses timestamps, filters and flags payment events within a specific timeframe, and selects relevant columns.

The processed data is then saved as a parquet file in S3 and the DataFrame is unloaded from memory.

Finally, it reads the processed data from the parquet file and displays a few rows.



In [17]:

flag_min_datetime = datetime.datetime(2024, 3, 21, 0, 0, 0)

flag_max_datetime = datetime.datetime(2024, 4, 18, 23, 59, 59)

In [18]:
%%time
if PROC_DS:
    sdf = spark.read.option('escape','"').csv(f's3a://{BUCKET}/{files_mask}', header=True)
    sdf = sdf.withColumn('event_datetime', F.to_timestamp("event_datetime"))
    flag_min_datetime = datetime.datetime(2024, 3, 21, 0, 0, 0)
    flag_max_datetime = datetime.datetime(2024, 4, 18, 23, 59, 59)
    print(flag_min_datetime, flag_max_datetime)
    sdf = sdf.withColumn(
        'payment_event_flag',
        (
            (F.col('event_name').like('%Мои штрафы/Оплата/Завершили оплату%') |
            F.col('event_name').like('%Мои штрафы/Оплата/Платёж принят%')) &
            F.col('event_datetime').between(flag_min_datetime, flag_max_datetime)
        ).cast("int")
    )
    sdf = sdf.select(
        'profile_id',
        'event_datetime',
        'payment_event_flag',
        'event_name'
    )
    #sdf.repartition(1).write.parquet(file_path_ds)
    sdf.unpersist()
sdf = spark.read.parquet(file_path_ds)
sdf.limit(5).toPandas()



if not PROC_DS:
    # Code to execute if PROC_DS is False
    flag_min_datetime = datetime.datetime(2024, 3, 21, 0, 0, 0)
    flag_max_datetime = datetime.datetime(2024, 4, 18, 23, 59, 59)
    print("flag min datetime: ", flag_min_datetime, '\n',
          "flag max datetime: ", flag_max_datetime)

2024-03-21 00:00:00 2024-04-18 23:59:59
CPU times: user 16.5 ms, sys: 17.7 ms, total: 34.2 ms
Wall time: 11 s


In [19]:
sdf.groupBy('payment_event_flag').count().show()

+------------------+---------+
|payment_event_flag|    count|
+------------------+---------+
|                 0|752434695|
|                 1|    45742|
+------------------+---------+



## 2. Dataset


This section defines variables for data processing and file paths, then performs several data processing stages:
Variables:

* `VER`: Version identifier.
* `PROC_DS`, `PROC_LAGS`, `PROC_VECS`: Flags to control data processing stages.
* `FRAC_0`: Fraction of negative examples to sample when processing lags.
* `BUCKET`: S3 bucket name.
* `files_path`, `files_mask`: Local paths and masks for raw data files.
* `file_path_ds`, `file_path_lags`, `file_path_trn`, `file_path_tst`: S3 paths for different stages of processed data.


• `VER = 'v2'`: This defines the version of your data processing pipeline. You might increment this version number when you make significant changes to the processing steps.
• `BUCKET` = 'pvc-84ea79a0-dc20-4a2d-86ab-f83c1f8d4a7b': This specifies the name of the bucket where your data is stored (likely on a cloud storage platform like AWS S3 or Yandex Object Storage).

**Flags:**

• `PROC_DS` (False): This flag controls whether to process the raw dataset. If True, the code will read the raw CSV files, extract relevant columns, filter by date, and create the data_raw.parquet file.

* Change to True: When you have new raw data or need to reprocess the existing raw data due to changes in the extraction logic.
* Keep as False: When you already have a processed data_raw.parquet file and don't need to re-process it.

• `PROC_LAGS` (False): This flag controls whether to process and create lag features. If True, the code will calculate lag features based on event history for different time windows and store them in the data_lags.parquet file.

* Change to True: When you need to recalculate lag features, such as when you've changed the time window definitions or added new events.
* Keep as False: When you already have the desired lag features in data_lags.parquet and don't need to recompute them.

• `FRAC_0` (.001): This variable sets the sampling fraction for events with payment_event_flag = 0 when processing lags. This is used to reduce the size of the dataset for faster processing while maintaining a representative sample.

* Adjust Value: You might change this value depending on the size of your dataset and the desired balance between processing time and data representativeness.

• `PROC_VECS` (True): This flag controls whether to vectorize the data using TF-IDF. If True, the code will transform the lag features into numerical vectors using TF-IDF and store them in data_vec_train.parquet and data_vec_test.parquet files.

* Change to False: If you want to experiment with other vectorization methods or use the data in its raw form.
* Keep as True: When TF-IDF vectorization is the desired approach for your modeling tasks.

**File Paths:**

• `files_path`, `files_mask`: These define the location and pattern of the raw data files.
• `file_path_ds`, `file_path_lags`, `file_path_trn`, `file_path_tst`: These specify the storage locations for the processed datasets at different stages of the pipeline.

By understanding these flags and variables, you can control which parts of the data processing pipeline are executed, allowing for efficient experimentation and iteration.


In [20]:
# Init Weights and Biases to begin storing data

wandb.init(project="ray-diploma", config={
    "version": VER,
    "proc_ds": PROC_DS,
    "proc_lags": PROC_LAGS,
    "proc_vecs": PROC_VECS,
    "frac_0": FRAC_0,
    "min_datetime": flag_min_datetime,
    "max_datetime": flag_max_datetime,
    "current_user": current_user,
    "uuid": unique_identifier,

},
           mode="online",
           dir="/home/jovyan/")

wandb: Currently logged in as: st110923 (gsom-diploma-jap). Use `wandb login --relogin` to force relogin


In [21]:
settings = wandb.Settings()
print(settings)

<Settings {'_args': None, '_aws_lambda': False, '_cli_only_mode': None, '_code_path_local': None, '_colab': False, '_cuda': None, '_disable_meta': False, '_disable_service': False, '_disable_setproctitle': False, '_disable_stats': False, '_disable_update_check': None, '_disable_viewer': None, '_disable_machine_info': False, '_executable': None, '_extra_http_headers': None, '_file_stream_retry_max': 125, '_file_stream_retry_wait_min_seconds': 2.0, '_file_stream_retry_wait_max_seconds': 60.0, '_file_stream_timeout_seconds': 180.0, '_file_transfer_retry_max': 20, '_file_transfer_retry_wait_min_seconds': 2.0, '_file_transfer_retry_wait_max_seconds': 60.0, '_file_transfer_timeout_seconds': 0.0, '_flow_control_custom': False, '_flow_control_disabled': False, '_graphql_retry_max': 20, '_graphql_retry_wait_min_seconds': 2.0, '_graphql_retry_wait_max_seconds': 60.0, '_graphql_timeout_seconds': 30.0, '_internal_check_process': 8.0, '_internal_queue_timeout': 2.0, '_ipython': False, '_jupyter': T

In [22]:
# Weights and Biases logging here:

counts = sdf.groupBy('payment_event_flag').count().toPandas()
wandb.log({"data_count": counts})
print("**After Initial Data Loading**")
print(counts)

**After Initial Data Loading**
   payment_event_flag      count
0                   0  752434695
1                   1      45742


### 2.2. Load or preprocess data - `lags` stage

2.2. Load or Preprocess Data - lags Stage

The `dataset_lags` function defines window specifications for different time intervals (e.g., 10 minutes to 1 hour, 1 day to 3 days).

It then uses these windows to calculate the list of event names within each time interval for each profile, creating lag features.

If `PROC_LAGS` is True, the function samples the data based on the `payment_event_flag` and the specified fraction for negative examples.

The processed data with lag features is saved as a parquet file and unloaded from memory.

Finally, it reads the data with lags and displays the count of positive and negative examples.

Lags new implementation



In [23]:
def dataset_lags(sdf, shift=0):
    hour = 60 * 60
    day = 24 * 60 * 60

    w_10min_to_1week = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-7 * day + shift, -10 * 60 + shift))
    w_1week_to_2weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-14 * day + shift, -7 * day + shift))
    w_2weeks_to_3weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-21 * day + shift, -14 * day + shift))
    w_3weeks_to_4weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-28 * day + shift, -21 * day + shift))
    w_4weeks_to_5weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-35 * day + shift, -28 * day + shift))
    w_5weeks_to_6weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-42 * day + shift, -35 * day + shift))
    w_6weeks_to_7weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-49 * day + shift, -42 * day + shift))
    w_7weeks_to_8weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-56 * day + shift, -49 * day + shift))
    w_8weeks_to_9weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-63 * day + shift, -56 * day + shift))
    w_9weeks_to_10weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-70 * day + shift, -63 * day + shift))
    w_10weeks_to_11weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-77 * day + shift, -70 * day + shift))
    w_11weeks_to_12weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-84 * day + shift, -77 * day + shift))
    w_12weeks_to_13weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-91 * day + shift, -84 * day + shift))
    w_13weeks_to_14weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-98 * day + shift, -91 * day + shift))
    w_14weeks_to_15weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-105 * day + shift, -98 * day + shift))
    w_15weeks_to_16weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-112 * day + shift, -105 * day + shift))
    w_16weeks_to_17weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-119 * day + shift, -112 * day + shift))
    w_17weeks_to_18weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-126 * day + shift, -119 * day + shift))
    w_18weeks_to_19weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-133 * day + shift, -126 * day + shift))
    w_19weeks_to_20weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-140 * day + shift, -133 * day + shift))
    w_20weeks_to_21weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-147 * day + shift, -140 * day + shift))
    w_21weeks_to_22weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-154 * day + shift, -147 * day + shift))
    w_22weeks_to_23weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-161 * day + shift, -154 * day + shift))
    w_23weeks_to_24weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-168 * day + shift, -161 * day + shift))
    w_24weeks_to_25weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-175 * day + shift, -168 * day + shift))
    w_25weeks_to_26weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-182 * day + shift, -175 * day + shift))
    w_26weeks_to_27weeks = (Window()
          .partitionBy(F.col('profile_id'))
          .orderBy(F.col('event_datetime').cast('timestamp').cast('long'))
          .rangeBetween(-189 * day + shift, -182 * day + shift))

    return (
        sdf
            #.withColumn('lag_10min_to_1week', F.collect_list('event_name').over(w_10min_to_1week))
            .withColumn('lag_1week_to_2weeks', F.collect_list('event_name').over(w_1week_to_2weeks))
            .withColumn('lag_2weeks_to_3weeks', F.collect_list('event_name').over(w_2weeks_to_3weeks))
            .withColumn('lag_3weeks_to_4weeks', F.collect_list('event_name').over(w_3weeks_to_4weeks))
            .withColumn('lag_4weeks_to_5weeks', F.collect_list('event_name').over(w_4weeks_to_5weeks))
            .withColumn('lag_5weeks_to_6weeks', F.collect_list('event_name').over(w_5weeks_to_6weeks))
            .withColumn('lag_6weeks_to_7weeks', F.collect_list('event_name').over(w_6weeks_to_7weeks))
            .withColumn('lag_7weeks_to_8weeks', F.collect_list('event_name').over(w_7weeks_to_8weeks))
            .withColumn('lag_8weeks_to_9weeks', F.collect_list('event_name').over(w_8weeks_to_9weeks))
            .withColumn('lag_9weeks_to_10weeks', F.collect_list('event_name').over(w_9weeks_to_10weeks))
            .withColumn('lag_10weeks_to_11weeks', F.collect_list('event_name').over(w_10weeks_to_11weeks))
            .withColumn('lag_11weeks_to_12weeks', F.collect_list('event_name').over(w_11weeks_to_12weeks))
            .withColumn('lag_12weeks_to_13weeks', F.collect_list('event_name').over(w_12weeks_to_13weeks))
            .withColumn('lag_13weeks_to_14weeks', F.collect_list('event_name').over(w_13weeks_to_14weeks))
            .withColumn('lag_14weeks_to_15weeks', F.collect_list('event_name').over(w_14weeks_to_15weeks))
            .withColumn('lag_15weeks_to_16weeks', F.collect_list('event_name').over(w_15weeks_to_16weeks))
            .withColumn('lag_16weeks_to_17weeks', F.collect_list('event_name').over(w_16weeks_to_17weeks))
            .withColumn('lag_17weeks_to_18weeks', F.collect_list('event_name').over(w_17weeks_to_18weeks))
            .withColumn('lag_18weeks_to_19weeks', F.collect_list('event_name').over(w_18weeks_to_19weeks))
            .withColumn('lag_19weeks_to_20weeks', F.collect_list('event_name').over(w_19weeks_to_20weeks))
            .withColumn('lag_20weeks_to_21weeks', F.collect_list('event_name').over(w_20weeks_to_21weeks))
            .withColumn('lag_21weeks_to_22weeks', F.collect_list('event_name').over(w_21weeks_to_22weeks))
            .withColumn('lag_22weeks_to_23weeks', F.collect_list('event_name').over(w_22weeks_to_23weeks))
            .withColumn('lag_23weeks_to_24weeks', F.collect_list('event_name').over(w_23weeks_to_24weeks))
            .withColumn('lag_24weeks_to_25weeks', F.collect_list('event_name').over(w_24weeks_to_25weeks))
            .withColumn('lag_25weeks_to_26weeks', F.collect_list('event_name').over(w_25weeks_to_26weeks))
            .withColumn('lag_26weeks_to_27weeks', F.collect_list('event_name').over(w_26weeks_to_27weeks))
            .select(
                'profile_id',
                'event_datetime',
                'payment_event_flag',
                'event_name',
                #'lag_10min_to_1week',
                'lag_1week_to_2weeks',
                'lag_2weeks_to_3weeks',
                'lag_3weeks_to_4weeks',
                'lag_4weeks_to_5weeks',
                'lag_5weeks_to_6weeks',
                'lag_6weeks_to_7weeks',
                'lag_7weeks_to_8weeks',
                'lag_8weeks_to_9weeks',
                'lag_9weeks_to_10weeks',
                'lag_10weeks_to_11weeks',
                'lag_11weeks_to_12weeks',
                'lag_12weeks_to_13weeks',
                'lag_13weeks_to_14weeks',
                'lag_14weeks_to_15weeks',
                'lag_15weeks_to_16weeks',
                'lag_16weeks_to_17weeks',
                'lag_17weeks_to_18weeks',
                'lag_18weeks_to_19weeks',
                'lag_19weeks_to_20weeks',
                'lag_20weeks_to_21weeks',
                'lag_21weeks_to_22weeks',
                'lag_22weeks_to_23weeks',
                'lag_23weeks_to_24weeks',
                'lag_24weeks_to_25weeks',
                'lag_25weeks_to_26weeks',
                'lag_26weeks_to_27weeks'
            )
        .orderBy(F.col('event_datetime'), ascending=False)
    )


Added the following code;

*     clean_parquet(file_path_lags)
*     dates  = (flag_min_datetime, flag_max_datetime)

1. `clean_parquet(file_path_lags)`:

This line of code is used to clean or remove any existing parquet files at the specified file_path_lags location before writing new data.

By calling `clean_parquet(file_path_lags)` before writing the new data with time lag windows, it ensures that any previous data stored at the same location is removed, preventing any conflicts or data inconsistencies.

**we have commented it out for this because we actually want to reuse the existing data when training and grid searching our models**

2. `dates = (flag_min_datetime, flag_max_datetime):`

This line creates a tuple named dates that contains two datetime values: flag_min_datetime and flag_max_datetime.

The purpose of dates = (flag_min_datetime, flag_max_datetime) is to create a tuple that represents a date range for filtering the data. The values of flag_min_datetime and flag_max_datetime are defined earlier in the notebook – I.e.

"""flag_min_datetime = datetime.datetime(2023, 8, 1, 0, 0, 0)
flag_max_datetime = datetime.datetime(2023, 8, 31, 23, 59, 59)
print(flag_min_datetime, flag_max_datetime)"""

After creating the dates tuple, the code uses it to filter the sdf DataFrame based on the event_datetime column.

The asterisk (*) before dates is used to unpack the tuple and pass the individual datetime values as arguments to the between function.


In [24]:
# Old version before VG update to filter for dates



if PROC_LAGS:
    sdf = sdf.sampleBy(
        'payment_event_flag',
        fractions={0: FRAC_0, 1: 1},
        seed=2023
    )
    sdf = dataset_lags(sdf)
    dates  = (flag_min_datetime, flag_max_datetime)
    sdf = sdf.filter(sdf.event_datetime.between(*dates))
    sdf = sdf.filter(
        #(F.size('lag_10min_to_1week')      > 0) |
        (F.size('lag_1week_to_2weeks')     > 0) |
        (F.size('lag_2weeks_to_3weeks')    > 0) |
        (F.size('lag_3weeks_to_4weeks')    > 0) |
        (F.size('lag_4weeks_to_5weeks')    > 0) |
        (F.size('lag_5weeks_to_6weeks')    > 0) |
        (F.size('lag_6weeks_to_7weeks')    > 0) |
        (F.size('lag_7weeks_to_8weeks')    > 0) |
        (F.size('lag_8weeks_to_9weeks')    > 0) |
        (F.size('lag_9weeks_to_10weeks')   > 0) |
        (F.size('lag_10weeks_to_11weeks')  > 0) |
        (F.size('lag_11weeks_to_12weeks')  > 0) |
        (F.size('lag_12weeks_to_13weeks')  > 0) |
        (F.size('lag_13weeks_to_14weeks')  > 0) |
        (F.size('lag_14weeks_to_15weeks')  > 0) |
        (F.size('lag_15weeks_to_16weeks')  > 0) |
        (F.size('lag_16weeks_to_17weeks')  > 0) |
        (F.size('lag_17weeks_to_18weeks')  > 0) |
        (F.size('lag_18weeks_to_19weeks')  > 0) |
        (F.size('lag_19weeks_to_20weeks')  > 0) |
        (F.size('lag_20weeks_to_21weeks')  > 0) |
        (F.size('lag_21weeks_to_22weeks')  > 0) |
        (F.size('lag_22weeks_to_23weeks')  > 0) |
        (F.size('lag_23weeks_to_24weeks')  > 0) |
        (F.size('lag_24weeks_to_25weeks')  > 0) |
        (F.size('lag_25weeks_to_26weeks')  > 0) |
        (F.size('lag_26weeks_to_27weeks')  > 0)
    )
    clean_parquet(file_path_lags)
    sdf.repartition(8).write.parquet(file_path_lags)
    sdf.unpersist()
sdf = spark.read.parquet(file_path_lags)


In [25]:
# Weights and Biases logging here:
counts = sdf.groupBy('payment_event_flag').count().toPandas()
wandb.log({"data_count_with_lags": counts})

Check your dataset has been reduced


In [26]:
print("**Dataset Size After Lag Feature Creation**", '/n', counts)

**Dataset Size After Lag Feature Creation** /n    payment_event_flag  count
0                   1  23109
1                   0  19594


In [27]:
sdf.printSchema()

root
 |-- profile_id: string (nullable = true)
 |-- event_datetime: timestamp (nullable = true)
 |-- payment_event_flag: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- lag_1week_to_2weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_2weeks_to_3weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_3weeks_to_4weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_4weeks_to_5weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_5weeks_to_6weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_6weeks_to_7weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_7weeks_to_8weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_8weeks_to_9weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_9weeks_to_1

### Train test split process

In [28]:
# Define train test split function

def stratified_split(sdf, frac, label, seed=2023):
    zeros = sdf.filter(sdf[label] == 0)
    ones = sdf.filter(sdf[label] == 1)
    train_, test_ = zeros.randomSplit([1 - frac, frac], seed=seed)
    train, test = ones.randomSplit([1 - frac, frac], seed=seed)
    train = train.union(train_)
    test = test.union(test_)
    return train, test

In [29]:
# Conduct train test split

sdf_train, sdf_test = stratified_split(
    sdf,
    frac=.2, # Size of the test dataset
    label='payment_event_flag',
    seed=2023
)

Check the dataset has been split into aproximately equal classes

In [30]:
sdf_train.groupBy('payment_event_flag').count().toPandas()

,payment_event_flag,count
0,1,18440
1,0,15649


In [31]:
sdf_test.groupBy('payment_event_flag').count().toPandas()

,payment_event_flag,count
0,1,4669
1,0,3945


In [32]:
# Weights and Biases logging here:

train_counts = sdf_train.groupBy('payment_event_flag').count().toPandas()
wandb.log({"train_data_count": train_counts})

test_counts = sdf_test.groupBy('payment_event_flag').count().toPandas()
wandb.log({"test_data_count": test_counts})

In [33]:
from pyspark.sql.functions import min, max

# Find the minimum and maximum dates
min_date = sdf.agg(min("event_datetime")).collect()[0][0]
max_date = sdf.agg(max("event_datetime")).collect()[0][0]

print(f"Minimum Date: {min_date}")
print(f"Maximum Date: {max_date}")

Minimum Date: 2024-03-21 00:00:17
Maximum Date: 2024-04-18 23:59:23


In [34]:
# Training set
train_min_date = sdf_train.agg(min("event_datetime")).collect()[0][0]
train_max_date = sdf_train.agg(max("event_datetime")).collect()[0][0]

print(f"Training Set Minimum Date: {train_min_date}")
print(f"Training Set Maximum Date: {train_max_date}")


Training Set Minimum Date: 2024-03-21 00:00:17
Training Set Maximum Date: 2024-04-18 23:57:47


In [35]:
# Test set
test_min_date = sdf_test.agg(min("event_datetime")).collect()[0][0]
test_max_date = sdf_test.agg(max("event_datetime")).collect()[0][0]

print(f"Test Set Minimum Date: {test_min_date}")
print(f"Test Set Maximum Date: {test_max_date}")

Test Set Minimum Date: 2024-03-21 00:05:21
Test Set Maximum Date: 2024-04-18 23:59:23


### 2.3. Load or preprocess data - `vectorize` stage

2.3. Load or Preprocess Data - vectorize Stage

This section defines a list of lag features to be used.

The datasets_tfidf function performs `TF-IDF` vectorization on the lag features for both training and test datasets.

It uses `HashingTF` to convert lists of event names into numerical feature vectors and `IDF` to rescale the features based on their document frequency.

The function also creates a dictionary mapping feature indices to the corresponding event names.

In [36]:
lags = [
#    'lag_10min_to_1week',
    'lag_1week_to_2weeks',
    'lag_2weeks_to_3weeks',
    'lag_3weeks_to_4weeks',
    'lag_4weeks_to_5weeks',
    'lag_5weeks_to_6weeks',
    'lag_6weeks_to_7weeks',
    'lag_7weeks_to_8weeks',
    'lag_8weeks_to_9weeks',
    'lag_9weeks_to_10weeks',
    'lag_10weeks_to_11weeks',
    'lag_11weeks_to_12weeks',
    'lag_12weeks_to_13weeks',
    'lag_13weeks_to_14weeks',
    'lag_14weeks_to_15weeks',
    'lag_15weeks_to_16weeks',
    'lag_16weeks_to_17weeks',
    'lag_17weeks_to_18weeks',
    'lag_18weeks_to_19weeks',
    'lag_19weeks_to_20weeks',
    'lag_20weeks_to_21weeks',
    'lag_21weeks_to_22weeks',
    'lag_22weeks_to_23weeks',
    'lag_23weeks_to_24weeks',
    'lag_24weeks_to_25weeks',
    'lag_25weeks_to_26weeks',
    'lag_26weeks_to_27weeks'
]

## TF-IDF implementation

In [37]:
def datasets_tfidf(sdf_train, sdf_test, lags, min_freq=3, num_features=10):
    features_dict = {}
    count = 0
    for lag in tqdm(lags):
        hashingTF = HashingTF(
            inputCol=lag,
            outputCol=lag + '_tf',
            numFeatures=num_features
        )
        featurizedData = hashingTF.transform(sdf_train)
        idf = IDF(
            inputCol=lag + '_tf',
            outputCol=lag + '_tfidf',
            minDocFreq=min_freq
        )
        idfModel = idf.fit(featurizedData)
        sdf_train = idfModel.transform(featurizedData)
        sdf_test = idfModel.transform(
            hashingTF.transform(sdf_test)
        )
        events = [
            x
            for xs in sdf_train.select(lag).distinct().rdd.flatMap(lambda x: x).collect()
            for x in xs
        ]
        hash_dict = {}
        for e in events:
            hash_dict[lag + '_' + e] = hashingTF.indexOf(e)
        for feat_num in range(num_features):
            tmp_list = []
            for k, v in hash_dict.items():
                if v == feat_num: tmp_list.append(k)
            features_dict[count * num_features + feat_num] = tmp_list
        count += 1
    return sdf_train, sdf_test, features_dict

In [38]:
# NEW version with clean_parquet (TF-IDF)

if PROC_VECS:
    sdf_train, sdf_test, features_dict = datasets_tfidf(
        sdf_train,
        sdf_test,
        lags,
        #vec_size=10,
        min_freq=3,
        num_features=100
    )
    clean_parquet(file_path_trn)
    sdf_train.repartition(8).write.parquet(file_path_trn)
    clean_parquet(file_path_tst)
    sdf_test.repartition(8).write.parquet(file_path_tst)
    sdf_train.unpersist()
    sdf_test.unpersist()
sdf_train = spark.read.parquet(file_path_trn)
sdf_test = spark.read.parquet(file_path_tst)


if not PROC_VECS:
    sdf_train, sdf_test, features_dict = datasets_tfidf(
        sdf_train, 
        sdf_test, 
        lags, 
        min_freq=3,
        num_features=100
    )
    print(len(features_dict.items()))


  0%|          | 0/26 [00:00<?, ?it/s]

In [39]:
# Check data size after reloading
print("**Training Set After TF-IDF**")
sdf_train.groupBy('payment_event_flag').count().show()
print("**Testing Set After TF-IDF**")
sdf_test.groupBy('payment_event_flag').count().show()

**Training Set After TF-IDF**
+------------------+-----+
|payment_event_flag|count|
+------------------+-----+
|                 1|18440|
|                 0|15649|
+------------------+-----+

**Testing Set After TF-IDF**
+------------------+-----+
|payment_event_flag|count|
+------------------+-----+
|                 1| 4669|
|                 0| 3945|
+------------------+-----+



In [40]:
sdf_train.printSchema()

root
 |-- profile_id: string (nullable = true)
 |-- event_datetime: timestamp (nullable = true)
 |-- payment_event_flag: integer (nullable = true)
 |-- event_name: string (nullable = true)
 |-- lag_1week_to_2weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_2weeks_to_3weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_3weeks_to_4weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_4weeks_to_5weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_5weeks_to_6weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_6weeks_to_7weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_7weeks_to_8weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_8weeks_to_9weeks: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- lag_9weeks_to_1

## 3. Model

### 3.1. Features assembling

3.1. Features Assembling

The features_assembled function prepares the data for model training:

* It selects the `TF-IDF` features and the target variable (`payment_event_flag`).
* It uses VectorAssembler to combine the `TF-IDF` features into a single vector column named `features`.
* It returns a DataFrame with the target variable and the assembled feature vector.

The upsampled function can be used to address class imbalance:

* It separates the data into positive and negative examples.
* It duplicates the positive examples to achieve a balanced class distribution based on the `UPSAMPLE` setting.

The code then:
* Defines a list of lag features to be used.
* Assembles features for both training and test sets.
* Optionally performs **upsampling** on the training set (and potentially the test set) if `UPSAMPLE` is enabled.
* Displays the class distribution after upsampling.

In [41]:
def features_assembled(sdf, feats):
    cols_to_model = [x + '_tfidf' for x in feats]
    cols_to_model.extend(['payment_event_flag'])
    print('columns to model:', cols_to_model)
    vecAssembler = VectorAssembler(
        inputCols=[c for c in cols_to_model if c != 'payment_event_flag'], 
        outputCol='features'
    )
    features = sdf.select(cols_to_model)
    features_vec = vecAssembler.transform(features)
    features_data = features_vec.select('payment_event_flag', 'features')
    return features_data

def upsampled(sdf, label, upsample='max'):
    zeros = sdf.filter(sdf[label] == 0)
    ones = sdf.filter(sdf[label] == 1)
    res = zeros.union(ones)
    if upsample == 'max':
        up_count = int(zeros.count() / ones.count())
        for _ in range(up_count - 1):
            res = res.union(ones)
    else:
        for _ in range(upsample - 1):
            res = res.union(ones)
    return res

In [42]:
# "MAX" Strategy: Setting UPSAMPLE = 'max' instructs the upsampled function to duplicate the minority class examples
# They are upsampled until their count becomes equal to the count of the majority class. 
# In other words, it aims for a 1:1 class ratio.

UPSAMPLE = 'max' # Can be either 'none' or 'max'


In [43]:
# Setting feats, make sure to comment out (#) any lags we will not use for train/pred 

feats = [
#    'lag_10min_to_1week',
    'lag_1week_to_2weeks',
    'lag_2weeks_to_3weeks',
    'lag_3weeks_to_4weeks',
    'lag_4weeks_to_5weeks',
    'lag_5weeks_to_6weeks',
    'lag_6weeks_to_7weeks',
    'lag_7weeks_to_8weeks',
    'lag_8weeks_to_9weeks',
    'lag_9weeks_to_10weeks',
    'lag_10weeks_to_11weeks',
    'lag_11weeks_to_12weeks',
    'lag_12weeks_to_13weeks',
    'lag_13weeks_to_14weeks',
    'lag_14weeks_to_15weeks',
    'lag_15weeks_to_16weeks',
    'lag_16weeks_to_17weeks',
    'lag_17weeks_to_18weeks',
    'lag_18weeks_to_19weeks',
    'lag_19weeks_to_20weeks',
    'lag_20weeks_to_21weeks',
    'lag_21weeks_to_22weeks',
    'lag_22weeks_to_23weeks',
    'lag_23weeks_to_24weeks',
    'lag_24weeks_to_25weeks',
    'lag_25weeks_to_26weeks',
    'lag_26weeks_to_27weeks'
]



features_train = features_assembled(sdf_train, feats=feats)
features_test = features_assembled(sdf_test, feats=feats)

if UPSAMPLE:
    features_train = upsampled(
        features_train, 
        label='payment_event_flag', 
        upsample=UPSAMPLE
    )
    # Use to upsample test set
    #features_test = upsampled(
    #    features_test, 
    #    label='payment_event_flag', 
    #    upsample=UPSAMPLE
    #)

columns to model: ['lag_1week_to_2weeks_tfidf', 'lag_2weeks_to_3weeks_tfidf', 'lag_3weeks_to_4weeks_tfidf', 'lag_4weeks_to_5weeks_tfidf', 'lag_5weeks_to_6weeks_tfidf', 'lag_6weeks_to_7weeks_tfidf', 'lag_7weeks_to_8weeks_tfidf', 'lag_8weeks_to_9weeks_tfidf', 'lag_9weeks_to_10weeks_tfidf', 'lag_10weeks_to_11weeks_tfidf', 'lag_11weeks_to_12weeks_tfidf', 'lag_12weeks_to_13weeks_tfidf', 'lag_13weeks_to_14weeks_tfidf', 'lag_14weeks_to_15weeks_tfidf', 'lag_15weeks_to_16weeks_tfidf', 'lag_16weeks_to_17weeks_tfidf', 'lag_17weeks_to_18weeks_tfidf', 'lag_18weeks_to_19weeks_tfidf', 'lag_19weeks_to_20weeks_tfidf', 'lag_20weeks_to_21weeks_tfidf', 'lag_21weeks_to_22weeks_tfidf', 'lag_22weeks_to_23weeks_tfidf', 'lag_23weeks_to_24weeks_tfidf', 'lag_24weeks_to_25weeks_tfidf', 'lag_25weeks_to_26weeks_tfidf', 'lag_26weeks_to_27weeks_tfidf', 'payment_event_flag']
columns to model: ['lag_1week_to_2weeks_tfidf', 'lag_2weeks_to_3weeks_tfidf', 'lag_3weeks_to_4weeks_tfidf', 'lag_4weeks_to_5weeks_tfidf', 'lag_5w

In [44]:
features_train.groupBy('payment_event_flag').count().toPandas()

,payment_event_flag,count
0,0,15649
1,1,18440


In [45]:
features_test.groupBy('payment_event_flag').count().toPandas()

,payment_event_flag,count
0,1,4669
1,0,3945


In [46]:
features_train.limit(3).toPandas()

,payment_event_flag,features
0,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### 3.2. Training and evaluating

In [47]:
from pyspark.ml.classification import GBTClassifier


In [76]:
# Define model filepath to link predictions results with the actual CSV 
from pyspark.ml.classification import GBTClassifier, LogisticRegression, LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

gbt = GBTClassifier(labelCol="payment_event_flag", featuresCol="features")

file_path_pred = f's3a://BUCKET/work/vA1/preds/GBT/22.04.24/st110923/1.csv'
print(f"- Prediction File Path: {file_path_pred}")

- Prediction File Path: s3a://BUCKET/work/vA1/preds/GBT/22.04.24/st110923/1.csv


In [49]:
features_train.printSchema()

root
 |-- payment_event_flag: integer (nullable = true)
 |-- features: vector (nullable = true)



In [50]:
%%time
model = gbt.fit(features_train)

CPU times: user 33.9 ms, sys: 1.54 ms, total: 35.4 ms
Wall time: 14.6 s


In [51]:
predictions = model.transform(features_test)
payment_event_flag_preds = predictions.select('prediction', 'payment_event_flag')
metrics = BinaryClassificationMetrics(
    payment_event_flag_preds.rdd.map(
        lambda lines: [float(x) for x in lines]
    )
)
print('ROC AUC:', metrics.areaUnderROC)
print('Area under PR-curve:', metrics.areaUnderPR)

/opt/conda/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


ROC AUC: 0.6616269540406332
Area under PR-curve: 0.7618419991852285


In [52]:
# Weights and Biases logging here
wandb.log({"gbt_roc_auc": metrics.areaUnderROC})
wandb.log({"gbt_pr_auc": metrics.areaUnderPR})

In [54]:
#predictions = model.transform(features_test)

y_true = predictions.select('payment_event_flag').rdd.map(lambda x: x['payment_event_flag']).collect()
y_pred = predictions.select('prediction').rdd.map(lambda x: x['prediction']).collect()

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.57      0.92      0.70      3945
           1       0.86      0.40      0.55      4669

    accuracy                           0.64      8614
   macro avg       0.71      0.66      0.62      8614
weighted avg       0.72      0.64      0.62      8614



In [58]:
gbt_report = classification_report(y_true, y_pred, output_dict=True)

In [59]:
wandb.log({"gbt_report": gbt_report})

Hyperopt optimization

In [60]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials


In [62]:
# Define the space of hyperparameters to search
space = {
    #'maxDepth': hp.choice('maxDepth', [5, 10, 20]),
    #'maxBins': hp.choice('maxBins', [32, 64, 128]),
    'maxIter': hp.choice('maxIter', [10, 20, 50]),
    'stepSize': hp.uniform('stepSize', 0.01, 0.1),
    'minInstancesPerNode': hp.choice('minInstancesPerNode', [1, 2, 4]),
    'minInfoGain': hp.uniform('minInfoGain', 0.0, 0.1)
}

# Define the objective function to minimize
def objective(params):
    # Create a GBTClassifier with the current set of hyperparameters
    gbt = GBTClassifier(featuresCol='features',
                        labelCol='payment_event_flag',
                        seed=42,
                        #maxDepth=params['maxDepth'],
                        #maxBins=params['maxBins'],
                        maxIter=params['maxIter'],
                        stepSize=params['stepSize'],
                        minInstancesPerNode=params['minInstancesPerNode'],
                        minInfoGain=params['minInfoGain'])
    
    # Create a Pipeline with the GBTClassifier
    pipeline = Pipeline(stages=[gbt])
    
    # Fit the model
    model = pipeline.fit(features_train)
    
    # Make predictions
    predictions = model.transform(features_test)
    
    # Evaluate the model using AUC
    evaluator = BinaryClassificationEvaluator(labelCol='payment_event_flag', metricName='areaUnderROC')
    auc = evaluator.evaluate(predictions)

    # Hyperopt minimizes the objective function, so return 1 - AUC as a loss to minimize
    return {'loss': 1 - auc, 'status': STATUS_OK}

# Create a trials object to store the results
trials = Trials()

# Run the hyperparameter search using the Tree of Parzen Estimators (TPE) algorithm
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=100,  # Set to the number of evaluations you want to run
    trials=trials
)



100%|██████████| 100/100 [17:59<00:00, 10.79s/trial, best loss: 0.3016841117735536]


In [67]:
print("Best parameters found: ", best_params)

Best parameters found:  {'maxIter': 2, 'stepSize': 0.08809824834577663, 'minInstancesPerNode': 1, 'minInfoGain': 7.100061569996724e-05}


In [71]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

best_params = {
    'maxIter': 2,
    'stepSize':  0.08809824834577663,
    'minInstancesPerNode': 1,
    'minInfoGain':7.100061569996724e-05
}

# Create a GBTClassifier with the best set of hyperparameters
gbt_best = GBTClassifier(featuresCol='features',
                         labelCol='payment_event_flag',
                         seed=42,
                         maxDepth=10,
                         maxBins=32,
                         maxIter=best_params['maxIter'],
                         stepSize=best_params['stepSize'],
                         minInstancesPerNode=best_params['minInstancesPerNode'],
                         minInfoGain=best_params['minInfoGain'])

# Create a Pipeline with the GBTClassifier
pipeline_best = Pipeline(stages=[gbt_best])

# Fit the model on the entire dataset
model_best = pipeline_best.fit(features_train)

# Make predictions on the test set
predictions_best = model_best.transform(features_test)

# Evaluate the model using AUC
evaluator_best = BinaryClassificationEvaluator(labelCol='payment_event_flag', metricName='areaUnderROC')
auc_best = evaluator_best.evaluate(predictions_best)

print("AUC ROC of the best model: ", auc_best)

AUC ROC of the best model:  0.6729618623605091


In [72]:

# Create a GBTClassifier with the best set of hyperparameters
gbt_best = GBTClassifier(featuresCol='features',
                         labelCol='payment_event_flag',
                         seed=42,
                         maxDepth=10,
                         maxBins=32,
                         maxIter=2,
                         stepSize=0.08809824834577663,
                         minInstancesPerNode=1,
                         minInfoGain=7.100061569996724e-05)

# Create a Pipeline with the GBTClassifier
pipeline_best = Pipeline(stages=[gbt_best])

# Fit the model on the entire dataset
model_best = pipeline_best.fit(features_train)

# Make predictions on the test set
predictions_best = model_best.transform(features_test)

# Evaluate the model using AUC
evaluator_best = BinaryClassificationEvaluator(labelCol='payment_event_flag', metricName='areaUnderROC')
auc_best = evaluator_best.evaluate(predictions_best)

print("AUC ROC of the best model: ", auc_best)

AUC ROC of the best model:  0.6729618623605091


In [ ]:
#Log metrics and params to W&B
wandb.log({"roc_auc_hyperopt": auc_best, "params_hyperopt":  best_params})

In [78]:
# Train the best model

best_model = gbt_best.fit(features_train)

In [79]:
predictions = best_model.transform(features_test)
payment_event_flag_preds = predictions.select('prediction', 'payment_event_flag')
metrics = BinaryClassificationMetrics(
    payment_event_flag_preds.rdd.map(
        lambda lines: [float(x) for x in lines]
    )
)
print('ROC AUC:', metrics.areaUnderROC)
print('Area under PR-curve:', metrics.areaUnderPR)

/opt/conda/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


ROC AUC: 0.6550680390386013
Area under PR-curve: 0.7596454077166834


In [80]:
#predictions = model.transform(features_test)

y_true = predictions.select('payment_event_flag').rdd.map(lambda x: x['payment_event_flag']).collect()
y_pred = predictions.select('prediction').rdd.map(lambda x: x['prediction']).collect()

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.56      0.92      0.70      3945
           1       0.86      0.39      0.53      4669

    accuracy                           0.63      8614
   macro avg       0.71      0.66      0.62      8614
weighted avg       0.72      0.63      0.61      8614



In [81]:
# Evaluate the best model on the test set
predictions = best_model.transform(features_test)
evaluator = BinaryClassificationEvaluator(labelCol="payment_event_flag")
best_auc = evaluator.evaluate(predictions)

In [82]:
# Calculate classification report
y_true = predictions.select('payment_event_flag').rdd.map(lambda x: x['payment_event_flag']).collect()
y_pred = predictions.select('prediction').rdd.map(lambda x: x['prediction']).collect()
report = classification_report(y_true, y_pred)


In [83]:
# Calculate PR AUC
payment_event_flag_preds = predictions.select('probability', 'payment_event_flag')
metrics = BinaryClassificationMetrics(payment_event_flag_preds.rdd.map(lambda lp: (float(lp[0][1]), float(lp[1]))))
pr_auc = metrics.areaUnderPR


/opt/conda/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [84]:
print(f"ROC AUC: {best_auc}")
print(f"PR AUC: {pr_auc}")

ROC AUC: 0.6729618623605091
PR AUC: 0.7630205704423072


In [ ]:
# Log additional metrics to W&B
wandb.log({"best_model_classification_report": wandb.Html(report)})
wandb.log({"best_model_roc_auc": best_auc})
wandb.log({"best_model_pr_auc": pr_auc})

In [ ]:
wandb.finish()  # Finalize W&B run